In [409]:
%pip install -q python-binance==1.0.19
%pip install -q pandas==2.0.3
%pip install -q numpy==1.25.0
%pip install -q requests==2.31.0
%pip install -q keyboard==0.13.5
%reset -f

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [410]:
import pandas as pd
import numpy  as np
import datetime
import requests
import time
import keyboard

from IPython.display import clear_output

pd.options.display.float_format = '{:,.8f}'.format

In [411]:
api_key    = ''
api_secret = ''
headers    = {'X-MBX-APIKEY' : api_key}

In [412]:
def getSignatureHmac(params):
  import hmac
  import hashlib

  s = '&'.join([f'{param}={value}' for param, value in params.items()])

  h = hmac.new(api_secret.encode("utf-8"), s.encode("utf-8"), hashlib.sha256 ).hexdigest()

  params['signature']  = h

  return params

In [413]:
def getAssets():
  """
  Return the assets table with a some treatments; 
  """

  #Get table of Binance assets
  r = requests.get('https://api.binance.com/api/v3/exchangeInfo')
  assets = pd.json_normalize(r.json()['symbols'], "filters", ['symbol',	'status',	'baseAsset', 'quoteAsset', 'baseAssetPrecision'])
  assets = assets.replace('NaN', np.nan).replace('nan', np.nan)

  #SELECT
  columns = {
             'symbol'            : str
            ,'baseAsset'         : str
            ,'quoteAsset'        : str
            ,'filterType'        : str
            ,'status'            : str
            ,'minPrice'          : float
            ,'maxPrice'          : float
            ,'tickSize'          : float                                       
            ,'bidMultiplierUp'   : float
            ,'bidMultiplierDown' : float
            ,'askMultiplierUp'   : float
            ,'askMultiplierDown' : float
            ,'avgPriceMins'      : float
            ,'avgPriceMins'      : float
            ,'minQty'            : float
            ,'maxQty'            : float
            ,'stepSize'          : float
            ,'baseAssetPrecision': int
            }
  
  assets = assets[columns.keys()]

  #CONVERT
  assets = assets.astype(columns)

  #FILTER
  #Doc of Binance v3 API:
  #Only currencies being traded
  #The PRICE_FILTER defines the price rules for a symbol
  #The PERCENT_PRICE_BY_SIDE filter defines the valid range for the price based on the last price of the symbol.
  #The LOT_SIZE filter defines the quantity (aka "lots" in auction terms) rules for a symbol.
  assets = assets[(assets['status'] == "TRADING") & assets['filterType'].isin(["PRICE_FILTER", "LOT_SIZE", "PERCENT_PRICE_BY_SIDE"])]\
                 .drop(columns=['status', 'filterType'])\
                 .dropna(axis='columns', how='all')

  #GROUP
  assets = assets.groupby(['symbol', 'baseAsset', 'quoteAsset'], as_index=False).max()

  return assets

In [414]:
def getFees():
  """
  Return the fees table; 
  """
  columns = {
               'symbol'          : str
              ,'makerCommission' : float
              ,'takerCommission' : float
            }
  params = {'timestamp' : int(time.time() * 1000)}
  return pd.DataFrame(requests.get('https://api.binance.com/sapi/v1/asset/tradeFee', headers=headers, params=getSignatureHmac(params)).json()).astype(columns)

In [415]:
def getPrices():
  """
  Return the price table; 
  """
  columns = {
               'symbol': str
              ,'price' : float
            }
  return pd.DataFrame(requests.get('https://api.binance.com/api/v3/ticker/price').json()).astype(columns)

In [416]:
def getTickers():
  """
  Return the tickers table; 
  """
  columns = {
               'symbol'   : str
              ,'bidPrice' : float
              ,'bidQty'   : float
              ,'askPrice' : float
              ,'askQty'   : float
            }
  return pd.DataFrame(requests.get('https://api.binance.com/api/v3/ticker/bookTicker').json()).astype(columns)

In [417]:
def getArbritrage(assets, fees, prices):
    """
    Calculate the arbritage factor table:
    return table with the arbitrage factor, if arbFactor > 0 then it's possible arbritage otherwise not;

    getArbritrage(assets, fees, prices)
    Parameters:
        assets: pandas Dataframe, must contain the columns: symbol, baseAsset, quoteAsset
        fees:   pandas Dataframe, must contain the columns: symbol, takerCommission
        prices: pandas Dataframe:
                if table contains the column named 'price' then the calculation will be done with this column
                that represent the spot price. (TABLE getPrices())
                else if table contains two columns named 'askPrice' and 'bidPrice' then the calculation will
                be done with these columns that represent sell price offer and buy price offer. (TABLE getTickers())
    """

    #Treatment of args    
    c = prices.columns
    if  'price' in c:
        tpBuy = 'price'
        tpSel = 'price'
    elif 'askPrice' in c and 'bidPrice' in c:
        tpBuy = 'askPrice'
        tpSel = 'bidPrice'
    else:
        raise Exception("The table dont have price or (askPrice and bidPrice) columns.") 

    base = assets.merge(fees   , on='symbol', how='left')\
                 .merge(prices, on='symbol', how='left')
    
    operation = {
     'buy':{'symbol': 'symbol', 'baseAsset': 'end'  , 'quoteAsset': 'start', tpBuy: 'marketPrice', 'takerCommission': 'tax'}
    ,'sel':{'symbol': 'symbol', 'baseAsset': 'start', 'quoteAsset': 'end'  , tpSel: 'marketPrice', 'takerCommission': 'tax'}
    }
    
    buy = base.rename(columns=operation['buy'])[operation['buy'].values()]
    buy['op'] = 'buy'
    buy["marketPrice"] = np.power((buy["marketPrice"]),-1)

    sel = base.rename(columns=operation['sel'])[operation['sel'].values()]
    sel['op'] = 'sel'

    conc = pd.concat([buy, sel])

    #merge two times
    m =  conc.merge(conc, left_on='end', right_on='start', how='inner', suffixes=('_0', None)).query('start_0 != end')\
             .merge(conc, left_on='end', right_on='start', how='inner', suffixes=('_1', '_2')).query('start_1 != end_2 and start_0 == end_2')

    #Three operations, all with taxes, start with 1 unit of the first operation coin
    m['arbFactor'] = m['marketPrice_0']* (1-m['tax_0'])*m['marketPrice_1']* (1-m['tax_1'])*m['marketPrice_2']*(1-m['tax_2'])

    m = m\
        [['arbFactor', 'symbol_0', 'symbol_1', 'symbol_2', 'op_0', 'op_1', 'op_2']]\
        .sort_values('arbFactor', axis=0, ascending=False, ignore_index=True)

    return m

In [418]:
def startSearch(factor):

  a = getAssets()[['symbol', 'baseAsset', 'quoteAsset']]
  f = getFees()
  ar_std = pd.DataFrame()

  while True:
    try:
      start = time.time()

      df = getArbritrage(a, f, getTickers())
      ar = df[df['arbFactor'] > factor]
      hd = df.head(5)

      clear_output(wait=True)

      print('To stop keep key "q" pressed or stop the cell!')
      
      print('\nAssets that could be arbitraged with the factor greather than {}: {}'.format(factor ,len(ar_std.index)))
      if ar_std.empty == False:
        ar['time'] = datetime.datetime.now()
        ar_std = pd.concat([ar, ar_std])
        print(ar_std.to_string(index=False))

      print('\nAssets in the top 5')
      print(hd.to_string(index=False))

      print("\nseconds/loop: {}".format(time.time() - start))
    
    except:
      pass

    if keyboard.is_pressed("q"):        
        break

In [420]:
startSearch(1)

To stop keep key "q" pressed or stop the cell!

Assets that could be arbitraged if the factor greather than 1: 0

Assets in the top 5
 arbFactor  symbol_0  symbol_1  symbol_2 op_0 op_1 op_2
0.99990009 FDUSDUSDT  BUSDUSDT FDUSDBUSD  sel  buy  buy
0.99990009  BUSDUSDT FDUSDBUSD FDUSDUSDT  buy  buy  sel
0.99990009 FDUSDBUSD FDUSDUSDT  BUSDUSDT  buy  sel  buy
0.99980037  TUSDBUSD  TUSDUSDT  BUSDUSDT  buy  sel  buy
0.99980037  BUSDUSDT  TUSDBUSD  TUSDUSDT  buy  buy  sel

seconds/loop: 1.4593696594238281
